In [107]:
import csv
import json
from openai import OpenAI

In [24]:
client = OpenAI()

def generate_product_reviews(num_reviews):
    reviews = []
    prompt_text = f"Napíš {num_reviews} pozitívnych a negatívnych recenzií"
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that generates unique reviews of products in Slovak with maximum of 25 words and different lengths into csv with columns review_text and sentiment (1-positive, 0-negative)"},
            {"role": "user", "content": prompt_text}
        ],
    )

    text = completion.choices[0].message.content 
    reviews.append(text)
    return reviews


data = []
data.extend(generate_product_reviews(5)) 
# data.extend(generate_product_reviews(5))  

# print(data)
# with open('product_sentiment_analysis_data.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["review_text", "sentiment"]) 
#     writer.writerows(data)


['review_text,sentiment\n"Fantastický výrobok! Vysoko odporúčam.",1\n"Veľmi efektívny, splnil moje očakávania.",1\n"Skvelá kvalita, rýchle doručenie.",1\n"Žiadne negatíva, dokonalý produkt.",1\n"Nesplnil moje očakávania, sklamanie.",0\n"Horšia kvalita ako som čakal.",0\n"Zlá hodnota za peniaze, neodporúčam.",0\n"Doručenie oneskorené, sklamanie.",0\n"Nedostatočná efektivita, neodporúčam.",0\n"Menej kvalitný ako vyzerá, sklamanie.",0']


In [25]:
print(data)

['review_text,sentiment\n"Fantastický výrobok! Vysoko odporúčam.",1\n"Veľmi efektívny, splnil moje očakávania.",1\n"Skvelá kvalita, rýchle doručenie.",1\n"Žiadne negatíva, dokonalý produkt.",1\n"Nesplnil moje očakávania, sklamanie.",0\n"Horšia kvalita ako som čakal.",0\n"Zlá hodnota za peniaze, neodporúčam.",0\n"Doručenie oneskorené, sklamanie.",0\n"Nedostatočná efektivita, neodporúčam.",0\n"Menej kvalitný ako vyzerá, sklamanie.",0']


In [93]:
from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
  file=open("batchinput.jsonl", "rb"),
  purpose="batch"
)


In [94]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "reviews data "
    }
)

Batch(id='batch_9xbE0JfwCIIXIfXIm5ShZJqP', completion_window='24h', created_at=1714556299, endpoint='/v1/chat/completions', input_file_id='file-cyJSRbGnHx9kL0uwUg9oIsV1', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1714642699, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'reviews data '}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [139]:
client.batches.retrieve("batch_9xbE0JfwCIIXIfXIm5ShZJqP")

Batch(id='batch_9xbE0JfwCIIXIfXIm5ShZJqP', completion_window='24h', created_at=1714556299, endpoint='/v1/chat/completions', input_file_id='file-cyJSRbGnHx9kL0uwUg9oIsV1', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1714557538, error_file_id=None, errors=None, expired_at=None, expires_at=1714642699, failed_at=None, finalizing_at=1714557538, in_progress_at=1714556300, metadata={'description': 'reviews data '}, output_file_id='file-YSIhGaf310zV6RUXt1AnNwsj', request_counts=BatchRequestCounts(completed=4, failed=0, total=4))

In [101]:
content = client.files.content("file-YSIhGaf310zV6RUXt1AnNwsj").content # 10 reviews
# content = client.files.content("batch_qQoYeYrrJQFdjjASb6oRjHUM") # 1000 reviews

In [104]:
with open('data/gpt_reviews.jsonl', 'wb') as file:
    file.write(content)

In [105]:
results = []
with open('data/gpt_reviews.jsonl', 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [110]:
with open('data/gpt_3.5_reviews.csv', 'w') as file:
    for res in results[:5]:
        task_id = res['custom_id']
        # Getting index from task id
        index = task_id.split('-')[-1]
        result = res['response']['body']['choices'][0]['message']['content']
        print(result)
        print("\n\n----------------------------\n\n")

review_text,sentiment
Produkt je neuveriteľný, veľmi som spokojný. ,1
Bohužial, kvalita nie je taká ako som očakával.,0
Super výkon a skvelý dizajn. ,1
Veľmi sklamaný, nefunguje podľa očakávaní.,0
Jednoduché ovládanie a vysoká kvalita. ,1
Nevhodný produkt, nespĺňa moje požiadavky. ,0
Vynikajúca hodnota za peniaze, veľmi odporúčam. ,1
Skôr sklamaný, čakal som viac. ,0
Krásny produkt s úžasnými funkciami. ,1
Zlé spracovanie, rýchlo sa poškodzuje. ,0
Presne to, čo som potreboval. ,1
Vôbec nespĺňa očakávania, veľmi sklamaný. ,0
Lahký a praktický, veľká výhoda pre cestovanie. ,1
Veľmi nekvalitný produkt, rýchlo sa pokazil. ,0
Skvelá podpora zákazníkov a rýchle doručenie. ,1
Nesprávna veľkosť, neodpovedá popisu. ,0
Úžasný výkon a dlhá životnosť batérie. ,1
Nepoužiteľný, nefunguje správne. ,0
Moderný a stylový produkt, zákaznícka spokojnosť. ,1
Veľmi vysoká cena, nie je to také dobré. ,0
Dizajn je veľmi pekný a obľúbený. ,1
Funguje len občas, veľmi sklamaný. ,0
Ľahká inštalácia a jednoduché n

In [125]:
import pandas as pd
translated_tweets_df = pd.read_csv('data/translated_tweets.csv')
gpt4_reviews_df = pd.read_csv('data/GPT4_reviews.csv')
heureka_reviews_df = pd.read_json('data/reviews.json')

In [121]:
translated_tweets_df.head(10)

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,Je to tak smutné pre môjho APL priateľa..........
1,2,0,Sentiment140,Chýbal mi nový trailer...
2,3,1,Sentiment140,Omg je už 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. som bol u toh...
4,5,0,Sentiment140,"Myslím si, že mi BF podvádza na mňa!!!"
5,6,0,Sentiment140,Alebo sa len príliš obávam?
6,7,1,Sentiment140,ZľavyZľavyZľavyZľavyZľavyZľavyZľavyZľavyZľavyZ...
7,8,0,Sentiment140,Sunny Again Work Tomorrow (TV dnes večer)
8,9,1,Sentiment140,"dneska som ťa dostala do uniformy, už mi chýbaš"
9,10,1,Sentiment140,"Hmmmm... som zvedavý, ako sa ona moje číslo @-)"


In [122]:
translated_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138103 entries, 0 to 1138102
Data columns (total 4 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   ItemID           1138103 non-null  int64 
 1   Sentiment        1138103 non-null  int64 
 2   SentimentSource  1138103 non-null  object
 3   SentimentText    1138103 non-null  object
dtypes: int64(2), object(2)
memory usage: 34.7+ MB


In [133]:
# gpt4_reviews_df.drop('review_id', axis=1, inplace=True)
gpt4_reviews_df.head(10)

,review_text,sentiment
0,Tento produkt je úžasný a veľmi užitočný!,1
1,Som sklamaný z kvality tohto produktu.,0
2,"Výborne funguje, odporúčam!",1
3,"Nevydrží dlho, nie je to stojí za peniaze.",0
4,Excelentná kvalita za rozumnú cenu.,1
5,Produkt neprišiel včas a bol poškodený.,0
6,Veľmi spokojný s nákupom!,1
7,Hrozná zákaznícka podpora a produkt nezodpoved...,0
8,"Výborný nákup, produkt presne ako je popísaný.",1
9,"Úplne nevhodný, vrátil som ho.",0


In [123]:
gpt4_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226 entries, 0 to 1225
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    1226 non-null   int64 
 1   review_text  1226 non-null   object
 2   sentiment    1226 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 28.9+ KB


In [136]:
heureka_reviews_df.head(10)

,review_text,sentiment
0,Som spokojná s tovarom aj rýchlosťou doručenia...,NaN
1,"Kvalita ,rychlost",NaN
2,Doposiaľ najrýchlejšie dodanie tovaru,1.0
3,Neskora donaśka,0.0
4,Dopravca s nemožnosťou platby kartou,0.0
5,S obchodom som spokojný ale vyjadril som nespo...,NaN
6,Blizko,1.0
7,Rýchle dodanie tovaru. Som veľmi spokojná.,1.0
8,Rýchlosť doručenia,1.0
9,"Tovar mi prišiel otvorený, našťastie funguje v...",NaN
